In [18]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os
from pydantic import BaseModel

from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
import time
import asyncio
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.base import TaskResult

In [2]:

load_dotenv()
api_key=os.getenv("OPENAI_API_KEY")

model_client=OpenAIChatCompletionClient(model='gpt-4o',api_key=api_key)

## Define Agents

In [3]:
coder_agent = AssistantAgent(
    name="CODE_AGENT", 
    model_client=model_client, 
    system_message="You give code in python to solve complex DSA problems. Give under 100 words.",
    description='A DSA solver')


code_reviewer = AssistantAgent(
    name = 'CODE_REVIEWER',
    model_client=model_client,
    description='A Code Reviewer',
    system_message="You review the code given by the complex_dsa_solver and make sure it is optimized.Give under 10 words. If you feel that the code is fine, please say 'TERMINATE'"
)

code_editor = AssistantAgent(
    name = 'CODE_EDITOR',
    model_client=model_client,
    description='A Code editor',
    system_message="You make the code easy to understand and add comments wherever required.Give under 10 words"
)

## Form a team with Round Robin strategy

In [4]:
team = RoundRobinGroupChat(participants=[coder_agent, code_reviewer, code_editor], max_turns=3)

## Task Initiate 

In [5]:
result = await team.run(task="write a simple code in python to add 2 numbers")

In [6]:
result

TaskResult(messages=[TextMessage(id='431071e7-46ff-44bc-9e92-d713e59bd56a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 39, 56, 609059, tzinfo=datetime.timezone.utc), content='write a simple code in python to add 2 numbers', type='TextMessage'), TextMessage(id='f2ddd277-431e-4ea1-9b69-a818995161c5', source='CODE_AGENT', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=66), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 39, 58, 542854, tzinfo=datetime.timezone.utc), content='```python\ndef add_numbers(a, b):\n    return a + b\n\n# Example usage\nnum1 = 5\nnum2 = 3\nresult = add_numbers(num1, num2)\nprint("The sum is:", result)\n```\nThis function `add_numbers` takes two arguments and returns their sum.', type='TextMessage'), TextMessage(id='b5cc79f0-8921-43f1-86ee-88930a5a9958', source='CODE_REVIEWER', models_usage=RequestUsage(prompt_tokens=136, completion_tokens=3), metadata={}, created_at=datetime.datetime(2025, 

In [ ]:
TaskResult(messages=[
    
    TextMessage(id='431071e7-46ff-44bc-9e92-d713e59bd56a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 39, 56, 609059, tzinfo=datetime.timezone.utc), content='write a simple code in python to add 2 numbers', type='TextMessage'), 
    
    TextMessage(id='f2ddd277-431e-4ea1-9b69-a818995161c5', source='CODE_AGENT', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=66), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 39, 58, 542854, tzinfo=datetime.timezone.utc), content='```python\ndef add_numbers(a, b):\n    return a + b\n\n# Example usage\nnum1 = 5\nnum2 = 3\nresult = add_numbers(num1, num2)\nprint("The sum is:", result)\n```\nThis function `add_numbers` takes two arguments and returns their sum.', type='TextMessage'), 
    
    TextMessage(id='b5cc79f0-8921-43f1-86ee-88930a5a9958', source='CODE_REVIEWER', models_usage=RequestUsage(prompt_tokens=136, completion_tokens=3), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 39, 59, 245421, tzinfo=datetime.timezone.utc), content='TERMINATE', type='TextMessage'), 
    
    TextMessage(id='47a0138e-2f7c-45d5-aca7-ead248aaf5e5', source='CODE_EDITOR', models_usage=RequestUsage(prompt_tokens=126, completion_tokens=88), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 40, 1, 806235, tzinfo=datetime.timezone.utc), content='```python\ndef add_numbers(a, b):\n    return a + b  # Return sum of a and b\n\n# Example usage\nnum1 = 5  # First number\nnum2 = 3  # Second number\nresult = add_numbers(num1, num2)  # Call function\nprint("The sum is:", result)  # Output result\n```\n\nThis code defines a function to add two numbers.', type='TextMessage')], 
    
    stop_reason='Maximum number of turns 3 reached.')

In [7]:
async def run_team():
    task = TextMessage(content='write a simple code in python to add 2 numbers',source='user')

    result = await team.run(task=task)

    print(result)

await run_team()

messages=[TextMessage(id='47f1c6cc-16ca-47af-9224-0d2c8f647dda', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 41, 58, 38785, tzinfo=datetime.timezone.utc), content='write a simple code in python to add 2 numbers', type='TextMessage'), TextMessage(id='421b8f25-d126-4075-841d-1d024a19f8d3', source='CODE_AGENT', models_usage=RequestUsage(prompt_tokens=234, completion_tokens=66), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 41, 59, 656857, tzinfo=datetime.timezone.utc), content='```python\ndef add_numbers(a, b):\n    return a + b\n\n# Example usage\nnum1 = 7\nnum2 = 4\nresult = add_numbers(num1, num2)\nprint("The sum is:", result)\n```\nThis function `add_numbers` takes two numbers and returns their sum.', type='TextMessage'), TextMessage(id='05532ee0-437a-4925-b8b1-9264e48a7019', source='CODE_REVIEWER', models_usage=RequestUsage(prompt_tokens=327, completion_tokens=3), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 42,

## With Single terminate condition

#### Example 1

In [10]:
termination_condition = TextMentionTermination(text="TERMINATE")

team = RoundRobinGroupChat(participants=[coder_agent, code_reviewer, code_editor], max_turns=3, termination_condition=termination_condition)

In [11]:
async def run_team():
    task = TextMessage(content='write a simple code in python to add 2 numbers',source='user')

    result = await team.run(task=task)

    print(result)

await run_team()

messages=[TextMessage(id='25594c83-d812-41d4-81fe-92fd525bbac9', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 43, 37, 686376, tzinfo=datetime.timezone.utc), content='write a simple code in python to add 2 numbers', type='TextMessage'), TextMessage(id='f268f962-c748-40eb-9759-226552ad8f77', source='CODE_AGENT', models_usage=RequestUsage(prompt_tokens=320, completion_tokens=69), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 43, 39, 184660, tzinfo=datetime.timezone.utc), content='```python\ndef add_numbers(a, b):\n    return a + b\n\n# Example usage\nnum1 = 10\nnum2 = 15\nresult = add_numbers(num1, num2)\nprint("The sum is:", result)\n```\n\nThis code defines a function `add_numbers` that returns the sum of two numbers.', type='TextMessage'), TextMessage(id='12d4cbad-da5c-4d24-b68d-f5081f71e629', source='CODE_REVIEWER', models_usage=RequestUsage(prompt_tokens=426, completion_tokens=3), metadata={}, created_at=datetime.datetime(202

In [ ]:
messages=[
    
    TextMessage(id='25594c83-d812-41d4-81fe-92fd525bbac9', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 43, 37, 686376, tzinfo=datetime.timezone.utc), content='write a simple code in python to add 2 numbers', type='TextMessage'), 
    
    TextMessage(id='f268f962-c748-40eb-9759-226552ad8f77', source='CODE_AGENT', models_usage=RequestUsage(prompt_tokens=320, completion_tokens=69), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 43, 39, 184660, tzinfo=datetime.timezone.utc), content='```python\ndef add_numbers(a, b):\n    return a + b\n\n# Example usage\nnum1 = 10\nnum2 = 15\nresult = add_numbers(num1, num2)\nprint("The sum is:", result)\n```\n\nThis code defines a function `add_numbers` that returns the sum of two numbers.', type='TextMessage'), 
    
    TextMessage(id='12d4cbad-da5c-4d24-b68d-f5081f71e629', source='CODE_REVIEWER', models_usage=RequestUsage(prompt_tokens=426, completion_tokens=3), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 43, 39, 819259, tzinfo=datetime.timezone.utc), content='TERMINATE', type='TextMessage')] 
    
    stop_reason="Text 'TERMINATE' mentioned"

#### Example 2

In [12]:
async def run_team():
    task = TextMessage(content='write a code to find median of two sorted arrays',source='user')

    result = await team.run(task=task)

    print(result)

await run_team()

messages=[TextMessage(id='ea73e866-1176-485d-8723-ecf4ed3493f2', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 45, 31, 943897, tzinfo=datetime.timezone.utc), content='write a code to find median of two sorted arrays', type='TextMessage'), TextMessage(id='3ed94ec1-21ff-4bb2-98a2-7daeaec018b1', source='CODE_EDITOR', models_usage=RequestUsage(prompt_tokens=529, completion_tokens=202), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 45, 35, 830014, tzinfo=datetime.timezone.utc), content='```python\ndef find_median_sorted_arrays(nums1, nums2):\n    combined = sorted(nums1 + nums2)  # Merge and sort arrays\n    length = len(combined)\n    \n    # If odd, return the middle element\n    if length % 2 == 1:\n        return combined[length // 2]\n    else:\n        # If even, return average of two middle elements\n        middle1 = combined[length // 2 - 1]\n        middle2 = combined[length // 2]\n        return (middle1 + middle2) / 2\n\n

In [ ]:
messages=[
    
    TextMessage(id='ea73e866-1176-485d-8723-ecf4ed3493f2', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 45, 31, 943897, tzinfo=datetime.timezone.utc), content='write a code to find median of two sorted arrays', type='TextMessage'), TextMessage(id='3ed94ec1-21ff-4bb2-98a2-7daeaec018b1', source='CODE_EDITOR', models_usage=RequestUsage(prompt_tokens=529, completion_tokens=202), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 45, 35, 830014, tzinfo=datetime.timezone.utc), content='```python\ndef find_median_sorted_arrays(nums1, nums2):\n    combined = sorted(nums1 + nums2)  # Merge and sort arrays\n    length = len(combined)\n    \n    # If odd, return the middle element\n    if length % 2 == 1:\n        return combined[length // 2]\n    else:\n        # If even, return average of two middle elements\n        middle1 = combined[length // 2 - 1]\n        middle2 = combined[length // 2]\n        return (middle1 + middle2) / 2\n\n# Example usage\narr1 = [1, 3]  # First sorted array\narr2 = [2, 4]  # Second sorted array\nmedian = find_median_sorted_arrays(arr1, arr2)\nprint("The median is:", median)  # Output median\n```\n\nThis code finds the median of two sorted arrays by combining, sorting, and selecting the middle element(s).', type='TextMessage'), 
    
    TextMessage(id='7d027748-404c-4ad7-8c19-61a8532ae44a', source='CODE_AGENT', models_usage=RequestUsage(prompt_tokens=629, completion_tokens=177), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 45, 38, 61403, tzinfo=datetime.timezone.utc), content='```python\ndef find_median_sorted_arrays(nums1, nums2):\n    combined = sorted(nums1 + nums2)  # Merge and sort arrays\n    length = len(combined)\n    \n    if length % 2 == 1:\n        return combined[length // 2]  # If odd, return the middle element\n    else:\n        middle1 = combined[length // 2 - 1]\n        middle2 = combined[length // 2]\n        return (middle1 + middle2) / 2  # If even, average of two middle elements\n\n# Example usage\narr1 = [1, 3]\narr2 = [2, 4]\nmedian = find_median_sorted_arrays(arr1, arr2)\nprint("The median is:", median)\n```\n\nThis code finds the median of two sorted arrays by merging and sorting them.', type='TextMessage'), 
    
    TextMessage(id='659cf51f-dfaa-45a9-b8a7-d4c05056fe52', source='CODE_REVIEWER', models_usage=RequestUsage(prompt_tokens=841, completion_tokens=3), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 45, 39, 114694, tzinfo=datetime.timezone.utc), content='TERMINATE', type='TextMessage')] 

stop_reason="Text 'TERMINATE' mentioned"

## With Multiple termination condition

In [15]:
termination_condition = TextMentionTermination(text="TERMINATE") | MaxMessageTermination(max_messages=4)

team1 = RoundRobinGroupChat(participants=[coder_agent, code_reviewer, code_editor], max_turns=10, termination_condition=termination_condition)

In [16]:
async def run_team():
    task = TextMessage(content='write a code to find median of two sorted arrays',source='user')

    result = await team1.run(task=task)

    print(result)

await run_team()

messages=[TextMessage(id='63a0eba0-6dfb-4645-996e-93683cbba05b', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 51, 6, 957675, tzinfo=datetime.timezone.utc), content='write a code to find median of two sorted arrays', type='TextMessage'), TextMessage(id='4cd72f88-33e2-459c-9a23-ae368571d09a', source='CODE_AGENT', models_usage=RequestUsage(prompt_tokens=825, completion_tokens=177), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 51, 10, 809153, tzinfo=datetime.timezone.utc), content='```python\ndef find_median_sorted_arrays(nums1, nums2):\n    combined = nums1 + nums2\n    combined.sort()  # Merge and sort the arrays\n    length = len(combined)\n    \n    if length % 2 == 1:\n        return combined[length // 2]  # Middle element for odd length\n    else:\n        mid1 = combined[length // 2 - 1]\n        mid2 = combined[length // 2]\n        return (mid1 + mid2) / 2  # Average of two middle elements for even length\n\n# Example usa

In [ ]:
messages=[
    
    TextMessage(id='63a0eba0-6dfb-4645-996e-93683cbba05b', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 51, 6, 957675, tzinfo=datetime.timezone.utc), content='write a code to find median of two sorted arrays', type='TextMessage'), 
    
    TextMessage(id='4cd72f88-33e2-459c-9a23-ae368571d09a', source='CODE_AGENT', models_usage=RequestUsage(prompt_tokens=825, completion_tokens=177), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 51, 10, 809153, tzinfo=datetime.timezone.utc), content='```python\ndef find_median_sorted_arrays(nums1, nums2):\n    combined = nums1 + nums2\n    combined.sort()  # Merge and sort the arrays\n    length = len(combined)\n    \n    if length % 2 == 1:\n        return combined[length // 2]  # Middle element for odd length\n    else:\n        mid1 = combined[length // 2 - 1]\n        mid2 = combined[length // 2]\n        return (mid1 + mid2) / 2  # Average of two middle elements for even length\n\n# Example usage\narr1 = [1, 3]\narr2 = [2, 4]\nmedian = find_median_sorted_arrays(arr1, arr2)\nprint("The median is:", median)\n```\n\nThis code merges and sorts two arrays, then finds the median.', type='TextMessage'), 
    
    TextMessage(id='fcbc9106-b91b-4af1-8ddf-cef3eef93a35', source='CODE_REVIEWER', models_usage=RequestUsage(prompt_tokens=1047, completion_tokens=3), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 51, 11, 523033, tzinfo=datetime.timezone.utc), content='TERMINATE', type='TextMessage')] 

stop_reason="Text 'TERMINATE' mentioned"

In [17]:
termination_condition = TextMentionTermination(text="TERMINATE") | MaxMessageTermination(max_messages=2)

team2 = RoundRobinGroupChat(participants=[coder_agent, code_reviewer, code_editor], max_turns=10, termination_condition=termination_condition)

async def run_team():
    task = TextMessage(content='write a code to find median of two sorted arrays',source='user')

    result = await team2.run(task=task)

    print(result)

await run_team()

messages=[TextMessage(id='ba579584-2c40-44ab-a268-1ffbad350781', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 52, 3, 177930, tzinfo=datetime.timezone.utc), content='write a code to find median of two sorted arrays', type='TextMessage'), TextMessage(id='e90e52de-9bfb-4921-a934-b07730fc5ae6', source='CODE_AGENT', models_usage=RequestUsage(prompt_tokens=1021, completion_tokens=380), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 52, 10, 295750, tzinfo=datetime.timezone.utc), content='```python\ndef find_median_sorted_arrays(nums1, nums2):\n    m, n = len(nums1), len(nums2)\n   \n    if m > n:  # Ensure nums1 is the smaller array\n        nums1, nums2, m, n = nums2, nums1, n, m\n\n    imin, imax, half_len = 0, m, (m + n + 1) // 2\n    while imin <= imax:\n        i = (imin + imax) // 2\n        j = half_len - i\n        if i < m and nums2[j-1] > nums1[i]:\n            imin = i + 1\n        elif i > 0 and nums1[i-1] > nums2[j]:\n    

In [ ]:
messages=[
    
    TextMessage(id='ba579584-2c40-44ab-a268-1ffbad350781', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 52, 3, 177930, tzinfo=datetime.timezone.utc), content='write a code to find median of two sorted arrays', type='TextMessage'), 

TextMessage(id='e90e52de-9bfb-4921-a934-b07730fc5ae6', source='CODE_AGENT', models_usage=RequestUsage(prompt_tokens=1021, completion_tokens=380), metadata={}, created_at=datetime.datetime(2025, 8, 14, 4, 52, 10, 295750, tzinfo=datetime.timezone.utc), content='```python\ndef find_median_sorted_arrays(nums1, nums2):\n    m, n = len(nums1), len(nums2)\n   \n    if m > n:  # Ensure nums1 is the smaller array\n        nums1, nums2, m, n = nums2, nums1, n, m\n\n    imin, imax, half_len = 0, m, (m + n + 1) // 2\n    while imin <= imax:\n        i = (imin + imax) // 2\n        j = half_len - i\n        if i < m and nums2[j-1] > nums1[i]:\n            imin = i + 1\n        elif i > 0 and nums1[i-1] > nums2[j]:\n            imax = i - 1\n        else:\n            if i == 0: max_of_left = nums2[j-1]\n            elif j == 0: max_of_left = nums1[i-1]\n            else: max_of_left = max(nums1[i-1], nums2[j-1])\n\n            if (m + n) % 2 == 1:\n                return max_of_left\n\n            if i == m: min_of_right = nums2[j]\n            elif j == n: min_of_right = nums1[i]\n            else: min_of_right = min(nums1[i], nums2[j])\n\n            return (max_of_left + min_of_right) / 2.0\n\n# Example usage\narr1 = [1, 3]\narr2 = [2, 4]\nmedian = find_median_sorted_arrays(arr1, arr2)\nprint("The median is:", median)\n```\n\nThis optimized approach uses binary search to find the median in O(log(min(m, n))) time when merging two sorted arrays.', type='TextMessage')] 

stop_reason='Maximum number of messages 2 reached, current message count: 2'

## How to run as stream

In [19]:
team3 = RoundRobinGroupChat(
    participants=[coder_agent, code_reviewer, code_editor],
    termination_condition=termination_condition,
    max_turns=6
)


async for message in team3.run_stream(task="Write a simple Hello world code ?"):  # type: ignore

    print(type(message))
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message.source,message)

<class 'autogen_agentchat.messages.TextMessage'>
user id='45f60713-28a2-4c24-b044-8e747e489358' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 14, 4, 53, 55, 433870, tzinfo=datetime.timezone.utc) content='Write a simple Hello world code ?' type='TextMessage'
<class 'autogen_agentchat.messages.TextMessage'>
CODE_AGENT id='7c03d9dc-fbcb-4146-ad28-d66489b6c6e1' source='CODE_AGENT' models_usage=RequestUsage(prompt_tokens=1417, completion_tokens=23) metadata={} created_at=datetime.datetime(2025, 8, 14, 4, 53, 56, 681945, tzinfo=datetime.timezone.utc) content='```python\nprint("Hello, world!")\n```\n\nThis code prints "Hello, world!" to the console.' type='TextMessage'
<class 'autogen_agentchat.base._task.TaskResult'>
Stop Reason: Maximum number of messages 2 reached, current message count: 2
